In [2]:
#importing important libraries and modules and defining dataframe
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('Advertising.csv')
print(df)

     Unnamed: 0     TV  Radio  Newspaper  Sales
0             1  230.1   37.8       69.2   22.1
1             2   44.5   39.3       45.1   10.4
2             3   17.2   45.9       69.3    9.3
3             4  151.5   41.3       58.5   18.5
4             5  180.8   10.8       58.4   12.9
..          ...    ...    ...        ...    ...
195         196   38.2    3.7       13.8    7.6
196         197   94.2    4.9        8.1    9.7
197         198  177.0    9.3        6.4   12.8
198         199  283.6   42.0       66.2   25.5
199         200  232.1    8.6        8.7   13.4

[200 rows x 5 columns]


In [3]:
#checking for null_values
null_values = df.isnull().sum()
#count null values for eachn column
print(null_values)
#as shown there are 24 null values in the Income Column

Unnamed: 0    0
TV            0
Radio         0
Newspaper     0
Sales         0
dtype: int64


In [4]:
#Define training Set
X = df.iloc[:, 1:4].values  # All columns except the last one
y = df.iloc[:, -1].values   # Only the last column

# Displaying the shapes of X and y to verify
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (200, 3)
Shape of y: (200,)


In [22]:
#we see that the features of X are in different ranges so we scale the features of X so they can be in the same range
scaler = StandardScaler()
x = scaler.fit_transform(X)
#splitting training set for model evaluation after fitting is done to check for bias(underfitting) or variance(overfitting)
from sklearn.model_selection import train_test_split

# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")


the shape of the training set (input) is: (120, 3)
the shape of the training set (target) is: (120,)

the shape of the cross validation set (input) is: (40, 3)
the shape of the cross validation set (target) is: (40,)

the shape of the test set (input) is: (40, 3)
the shape of the test set (target) is: (40,)


In [24]:
#increasing the training set size because the more training examples we have, the better our algorithm learns the training set
#so we duplicate the input and target array by a 1000 times
Xt = np.tile(x_train,(1000,1))
Yt= np.tile(y_train,(1000))   
print(Xt.shape, Yt.shape)   


(120000, 3) (120000,)


In [25]:
from sklearn.linear_model import SGDRegressor
#creating the regression model and defining the maximum number of iterations
sgdr = SGDRegressor(max_iter=1000)
#fitting the model to the train set
sgdr.fit(Xt, Yt)
#the library automatically defines the loss function and the learning rate which can be seen when we print out sgdr
print(sgdr.get_params())
#we also see it's number of iterations before it reached the gloal minimum i.e(where the cost function J(w,b) is minimum)
print(f"number of iterations completed: {sgdr.n_iter_}")

[[ 0.47592034 -0.14612098 -0.9691538 ]
 [-0.14879261 -1.24000266 -0.97836017]
 [ 1.08311798 -1.29402151  0.29211789]]
[15.6 11.2 12.3]
{'alpha': 0.0001, 'average': False, 'early_stopping': False, 'epsilon': 0.1, 'eta0': 0.01, 'fit_intercept': True, 'l1_ratio': 0.15, 'learning_rate': 'invscaling', 'loss': 'squared_loss', 'max_iter': 1000, 'n_iter_no_change': 5, 'penalty': 'l2', 'power_t': 0.25, 'random_state': None, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
number of iterations completed: 7


In [27]:
#printing out the parameters
#remember these are the parameters for the normalized X(input features)

b_norm = sgdr.intercept_
w_norm = sgdr.coef_
print(w_norm)
print(b_norm)

[3.80416486 2.70912258 0.05065966]
[14.02843866]


In [33]:
#making predictions with model parameters for the train,cv and test set
yhat_train = sgdr.predict(x_train)
yhat_cv = sgdr.predict(x_cv)
yhat_test = sgdr.predict(x_test)

#calculating how well the model works on the train set and the cross validation set by finding the MSE(how much the prediction)
#differs from the target for the train and validation set
from sklearn.metrics import mean_squared_error
err_train = mean_squared_error(y_train, yhat_train) / 2
err_cv = mean_squared_error(y_cv, yhat_cv) / 2
err_test = mean_squared_error(y_test, yhat_test) / 2
print(err_train)
print(err_cv)
print(err_test)


1.5179633031796171
0.8047747041013155
1.6549263901336626


In [ ]:
#so we see the model does relatively well for the train set and does even better for the cross-validation set. since there is
#no high bias or high variance we can say this model is good for prediction
#Note: for any new data that wants to use the model for prediction, this data has to be scaled 